# 텍스트 마이닝

일정한 길이의 vector로 변환

변환된 vector에 머신러닝/딥러닝 기법을 적용

##텍스트 마이닝 적용분야

document classification

document generation

keyword extraction

topic modeling




# 텍스트마이닝 도구- 파이썬

NLTK:가장 많이 알려진 NLP 라이브러리

Scikit Learn: 머신러닝 라이브러리

Gensim: Word2Vec으로 유명


# 텍스트 마이닝 기본도구

목적: document, sentence 등을 sparse vector로 변환
    
Tokenize: 대상이 되는 문서/문장을 최소 단위로 쪼갬
    
Text normalization: 최소단위를 표준화
    
Pos-tagging: 최소 의미단위로 나누어진 대상에 대해 품사를 부착
    
Chunking: pos-tagging의 결과를 명사구, 형용사구, 분사구 등과 같은 말모듬으로 다시 합치는 과정
    
Bow,TFIDF: tokenized결과를 이용하여 문서를 vector로 표현    

In [1]:
#Tokenize 예시
import nltk
from nltk.tokenize import word_tokenize
text="In this class, we use the  am actively looking for Ph.D. students. and you are a Ph.D. student."
print(word_tokenize(text))

['In', 'this', 'class', ',', 'we', 'use', 'the', 'am', 'actively', 'looking', 'for', 'Ph.D.', 'students', '.', 'and', 'you', 'are', 'a', 'Ph.D.', 'student', '.']


In [2]:
#text normalization-stemming-porter stemmer(어간 추출) 예시
from nltk.stem import PorterStemmer
ps = PorterStemmer()
ps

<PorterStemmer>

In [3]:
print("1. cooking :", ps.stem("cooking"))
print("2. cookery :", ps.stem("cookery"))
print("3. believes :", ps.stem("believes"))
print("4. using :", ps.stem("using"))

1. cooking : cook
2. cookery : cookeri
3. believes : believ
4. using : use


In [4]:
#lemmatization(표제어 추출)예시
from nltk.stem import WordNetLemmatizer
import nltk
lemmatizer = WordNetLemmatizer()

In [5]:
print("1-1. cooking :", lemmatizer.lemmatize("cooking"))
print("1-2. cooking(pos=\"v\") :", lemmatizer.lemmatize("cooking", pos="v"))
print("1-3. cooking(pos=\"n\") :", lemmatizer.lemmatize("cooking", pos="n"))
print("2. cookbooks :", lemmatizer.lemmatize("cookbooks"))
print("3. believes :", lemmatizer.lemmatize("believes", pos="v"))
print("4. buses :", lemmatizer.lemmatize("buses"))
print("5. using :", lemmatizer.lemmatize("using", pos="v"))

1-1. cooking : cooking
1-2. cooking(pos="v") : cook
1-3. cooking(pos="n") : cooking
2. cookbooks : cookbook
3. believes : believe
4. buses : bus
5. using : use


In [6]:
#pos-tagging 예시
tokens = "The little yellow dog barked at the Persian cat".split()
tags_en = nltk.pos_tag(tokens)
tags_en

[('The', 'DT'),
 ('little', 'JJ'),
 ('yellow', 'JJ'),
 ('dog', 'NN'),
 ('barked', 'VBD'),
 ('at', 'IN'),
 ('the', 'DT'),
 ('Persian', 'JJ'),
 ('cat', 'NN')]

In [ ]:
#chunking 예시
parser_en = nltk.RegexpParser("NP: {<DT>?<JJ>?<NN.*>*}")
chunks_en = parser_en.parse(tags_en)
chunks_en.draw()

# 개체명 인식
개체명은 기관, 단체, 사람, 날짜등과 같이 특정 정보에 해당하는 명사구를 의미

관계인식(Relation Extraction) 

NER에 의해 추출된 개체명들을 대상으로 그들간의 관계를 추출 하는 작업 

# BOW(Bag of Words)

Vector Space Model: 문서를 bag of words로 표현, 단어가 쓰여진 순서는 무시

BOW에 있어서 중요한 것은 단어의 등장 빈도

# TFIDF

단어의 count를 단어가 나타난 문서의 수로 나눠서 자주 등장하지 않는 단어의 weight를 올림

tf(d, t): 문서d에 단어t가 나타난 횟수, count vector와 동일, 로그스케일 등 다양한 변형이 있음
    
df(t): 전체 문서 중에서 단어t를 포함하는 문서의 수

idf(t): df(t)의 역수

In [8]:
import pandas as pd # 데이터프레임 사용을 위해
from math import log # IDF 계산을 위해

In [9]:
docs = [
  '먹고 싶은 사과',
  '먹고 싶은 바나나',
  '길고 노란 바나나 바나나',
  '저는 과일이 좋아요'
] 
vocab = list(set(w for doc in docs for w in doc.split()))
vocab.sort()

In [10]:
N = len(docs) # 총 문서의 수

def tf(t, d):
    return d.count(t)

def idf(t):
    df = 0
    for doc in docs:
        df += t in doc
    return log(N/(df + 1))

def tfidf(t, d):
    return tf(t,d)* idf(t)

In [6]:
result = []
for i in range(N): # 각 문서에 대해서 아래 명령을 수행
    result.append([])
    d = docs[i]
    for j in range(len(vocab)):
        t = vocab[j]        
        result[-1].append(tf(t, d))

tf_ = pd.DataFrame(result, columns = vocab)
tf_

,과일이,길고,노란,먹고,바나나,사과,싶은,저는,좋아요
0,0,0,0,1,0,1,1,0,0
1,0,0,0,1,1,0,1,0,0
2,0,1,1,0,2,0,0,0,0
3,1,0,0,0,0,0,0,1,1


In [7]:
result = []
for j in range(len(vocab)):
    t = vocab[j]
    result.append(idf(t))

idf_ = pd.DataFrame(result, index = vocab, columns = ["IDF"])
idf_

,IDF
과일이,0.693147
길고,0.693147
노란,0.693147
먹고,0.287682
바나나,0.287682
사과,0.693147
싶은,0.287682
저는,0.693147
좋아요,0.693147


# Text classification with BOW/TFIDF
Naive Bayes 나이브 베이즈

Logistic regression 로지스틱 회귀

Decision tree 의사결정 나무

# 한글 감성분석

label이 0이면 부정, 1이면 긍정


# 주성분 분석

선형결합

공분산 행렬

고유값이 큰 순서대로 고유벡터를 정렬하여 차원 선택

선택된 고유벡터와 X의 선형결합으로 차원 축소

In [18]:
import pandas as pd
url = "https://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data"
df = pd.read_csv(url, names=['sepal length','sepal width','petal length','petal width','target'])
df.head()

,sepal length,sepal width,petal length,petal width,target
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa


In [19]:
from sklearn.preprocessing import StandardScaler  # 표준화 패키지 라이브러리 
x = df.drop(['target'], axis=1).values # 독립변인들의 value값만 추출
y = df['target'].values # 종속변인 추출

x = StandardScaler().fit_transform(x) # x객체에 x를 표준화한 데이터를 저장

features = ['sepal length', 'sepal width', 'petal length', 'petal width']
pd.DataFrame(x, columns=features).head()

,sepal length,sepal width,petal length,petal width
0,-0.900681,1.032057,-1.341272,-1.312977
1,-1.143017,-0.124958,-1.341272,-1.312977
2,-1.385353,0.337848,-1.398138,-1.312977
3,-1.506521,0.106445,-1.284407,-1.312977
4,-1.021849,1.263460,-1.341272,-1.312977


In [20]:
from sklearn.decomposition import PCA
pca = PCA(n_components=2) # 주성분을 몇개로 할지 결정
printcipalComponents = pca.fit_transform(x)
principalDf = pd.DataFrame(data=printcipalComponents, columns = ['principal component1', 'principal component2'])
# 주성분으로 이루어진 데이터 프레임 구성

In [21]:
principalDf.head()

,principal component1,principal component2
0,-2.264542,0.505704
1,-2.086426,-0.655405
2,-2.367950,-0.318477
3,-2.304197,-0.575368
4,-2.388777,0.674767


In [22]:
pca.explained_variance_ratio_

array([0.72770452, 0.23030523])

# TOPIC MODELING 토픽 모델링

토픽모델링의 원리: 토픽은 주제를 의미하는 용어로 사용되며, 각 문서들이 특정한 주제에 속할 확률분포와 주제로부터 특정 단어들이 파생되어 나올 확률분호가 주어졌을 때, 이 두 확률분포를 조합하여 각 문서들에 들어가는 단어들의 확률분포를 계산


In [31]:
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation


In [35]:
# 주어진 데이터셋의 일부 카톄고리 데이터만 추출하므로 카테고리 사전에 설정 
cats = ['rec.motorcycles', 'rec.sport.baseball' , 
'comp.graphics' , 'comp.windows.x' , 
'talk.politics.mideast', 'soc.religion.christian', 'sci.electronics' , 'sci.me 
'sci.med'] 
# 설정해준 카테고리의 데이터들만 추출 
news_df = fetch_20newsgroups(subset='all' , remove=('headers','footers','quotes'),categories=cats,random_state=12)

# CountVectorizer로 텍스트 데이터들 단어 빈도수에 기반해 벡터화시키기 

count_vect= CountVectorizer(max_df=0.95, max_features=1000 , min_df=2, stop_words='english', ngram_range=(1,2))

ftr_vect=count_vect.fit_transform(news_df.data)

# LDA클래스를 이용해서 피처 벡터화시킨 것을 토픽모델링 시키기

# 8개의 주제만 뽑았으니 n components(토픽개수) 8로설정

lda = LatentDirichletAllocation(n_components=8, random_state=42)  

Ida.fit(ftr_vect) 
# components 속성은 8걔의 토픽별(row)로 1000개의 feature(단어)들의 분포수치를 보여줌
print (Ida.components_.shape) 
print (Ida.components_)


SyntaxError: EOL while scanning string literal (<ipython-input-35-465c18de9b03>, line 4)

# Word Embedding

단어에 대한 vector의 dimension reduction이 목표

one-hot-encoding

word embedding: one-hot-encoding으로 표현된 단어를 dense vector로 변환



In [36]:
from konlpy.tag import Okt  
okt=Okt()  
token=okt.morphs("나는 자연어 처리를 배운다")   #토큰화 수행
print(token)

['나', '는', '자연어', '처리', '를', '배운다']


In [37]:
word2index={} #각 토큰에 고유한 인덱스 부여
for voca in token:
     if voca not in word2index.keys():
       word2index[voca]=len(word2index)
print(word2index)

{'나': 0, '는': 1, '자연어': 2, '처리': 3, '를': 4, '배운다': 5}


In [38]:
def one_hot_encoding(word, word2index):
       one_hot_vector = [0]*(len(word2index)) #토큰을 입력하면 해당 토큰에 대한 one-hot-vector를 만들어내는 함수
       index=word2index[word]
       one_hot_vector[index]=1
       return one_hot_vector

In [39]:
one_hot_encoding("자연어",word2index) #함수에 '자연어'라는 토큰을 입력

[0, 0, 1, 0, 0, 0]

# Word2Vec

문장에 나타난 단어들의 순서를 이용해 word embedding을 수행

CBOW:주변 단어들을 이용하여 다음 단어를 예측

Skip-gram: 한 단어의 주변단어들을 예측


In [52]:
import gensim

sentences=[['first','second','third','fourth']]*1000 #각 단어를 문장 목록으로 설정

model1 = gensim.models.Word2Vec(sentences, min_count = 1, size = 2) #2개의 동일한 모델구현
model2 = gensim.models.Word2Vec(sentences, min_count = 1, size = 2)
print(all(model1['first']==model2['first']))


True


C:\Users\pmg09\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  import sys


In [53]:
model3 = gensim.models.Word2Vec(sentences, min_count = 1, size = 2, seed = 1234)
model4 = gensim.models.Word2Vec(sentences, min_count = 1, size = 2, seed = 1234)
print(all(model3['first']==model4['first']))

True


C:\Users\pmg09\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  This is separate from the ipykernel package so we can avoid doing imports until


# RBM
사전학습 목적으로 개발

Deep NN의 vanishing gradient 문제 해결을 위해 제안

#Autoencoder
RBM과 유사한 개념

# context의 파악

N-gram: 문맥을 파악하기 위한 전통적인 방법(bi-gram,tri-gram)
       
대상이 되는 문자열을 하나의 단어 단위가 아닌 2개 이상의 단위로 잘라서 처리

딥러닝-RNN: 문장을 단어들의 시퀀스 혹은 시리즈로 처리

# LSTM

RNN의 문제: 문장이 길수록 층이 깊은 형태를 갖게됨-->앞부분의 단어 정보가 학습되지 않음

LSTM: 직통 통로를 만들어 RNN의 문제를 해결

    

# CNN(합성곱 신경망)

CNN은 원래 이미 처리를 위해 개발된 신경망으로, 현재는 인간의 이미지 인식보다 더 나은 인식 성능을 보이고 있음

합성곱층과 풀링층으로 구성되며, 합성곱층은 2차원 이미지에서 특정 영역의 특징을 추출하는 역할을 한다. 이는 연속된 단어들의 특징을 추출하는 것과 유사한 특성이 있음.

# Sequence to Sequence

지금까지 입력은 sequence, 출력은 하나의 값인 경우가 일반적이었으나

번역, 챗봇, summarize 등은 출력도 sequence가 되어야 함

In [61]:
# 파이썬의 딕셔너리 자료형을 선언
# 키(Key) : 값(value)의 형식으로 키와 값의 쌍(Pair)을 선언한다.
dict = {"2017" : "Transformer", "2018" : "BERT"}

In [62]:
print(dict["2017"]) #2017이라는 키에 해당되는 값을 출력

Transformer


In [63]:
print(dict["2018"])  #2018이라는 키에 해당되는 값을 출력

BERT
